In [13]:
import pandas as pd

import numpy as np

import re

In [14]:
config_algo_names = ['CommonChars', 'ContainsSubstring', 'CountVowels','ReverseArray', 'BinarySearchStrings', 'Multiples','CrossSum', 'Swap',  'InsertionSort','GreatestCommonDivisor']

config_unique_code_variable = "PD14_01"

config_answer_variables = ['PT03_01', 'PT08_01', 'PT13_01', 'PT18_01', 'PT23_01','PT28_01', 'PT33_01', 'PT38_01', 'PT43_01', 'PT48_01']

config_difficulty_variables = ['PT04', 'PT09', 'PT14', 'PT19', 'PT24', 'PT30', 'PT34', 'PT39', 'PT44', 'PT49']

config_time_variables = ['TIME006','TIME008','TIME010','TIME012', 'TIME014','TIME016','TIME018','TIME020','TIME022', 'TIME024']

config_click_variables = ['PT01_01', 'PT06_01', 'PT11_01', 'PT16_01', 'PT21_01','PT26_01', 'PT31_01', 'PT36_01', 'PT41_01', 'PT46_01']

config_correctness_variables = ['^1$', '^(?i)true$', '^4$', '^210461$', '^1$', 
                                '^32$', '^8$', '^42$', '^12578$', '^4$']



config_datasheet = r'./pretest.xlsx'

In [15]:
raw = pd.read_excel(config_datasheet)
df = pd.DataFrame(raw)
df = df.dropna()

from scipy.stats import ttest_ind


In [16]:
df_matrix = []
raw = pd.read_excel(config_datasheet)

# load data
for row, _stuff in enumerate(config_algo_names):
    df_array = []
    df = pd.DataFrame(raw, columns = [config_unique_code_variable,
                                      config_answer_variables[row],
                                      config_correctness_variables[row],
                                      config_difficulty_variables[row],
                                      config_time_variables[row],
                                      config_click_variables[row],])
    #df = df.dropna()
    df_array.append(df)
    df_matrix.append(df_array)

In [17]:
cols = ["unique_code", "response_time", "correctness", "difficulty", "flag", "algo_name", "click_data"]

df_tmp = pd.DataFrame(columns=cols)


for row_idx, df_row in enumerate(df_matrix):
    algo_name = config_algo_names[row_idx]
    for col, df in enumerate(df_row): 
        
        for index, row in df.iterrows():
            unique_code = row["PD14_01"]
            response_time = int(row[config_time_variables[row_idx]])
            
            if row_idx == 3:
                answer = str(row[config_answer_variables[row_idx]])
                rep = {"{": "", "+": "", " ":"", ",":"", "[":"", "]":"", "}":""} # define desired replacements here

                rep = dict((re.escape(k), v) for k, v in rep.items()) 
                pattern = re.compile("|".join(rep.keys()))
                answer = pattern.sub(lambda m: rep[re.escape(m.group(0))], answer)
            elif row_idx == 8:
                answer = str(row[config_answer_variables[row_idx]])
                rep = {"{": "", "+": "", " ":"", ",":"", "[":"", "]":"", "}":""} # define desired replacements here
                rep = dict((re.escape(k), v) for k, v in rep.items()) 
                pattern = re.compile("|".join(rep.keys()))
                answer = pattern.sub(lambda m: rep[re.escape(m.group(0))], answer)
            else:
                answer = row[config_answer_variables[row_idx]]
            
            difficulty = row[config_difficulty_variables[row_idx]]
            click_data = row[config_click_variables[row_idx]]
            flag = ""
            
            correctness = bool(re.match(config_correctness_variables[row_idx], str(answer)))
            data = pd.DataFrame([[unique_code, response_time, correctness, difficulty, flag, algo_name, click_data]], columns=cols)
                  
            df_tmp = df_tmp.append(data)
#7nKTra Swap manuell richtig
#5iOPrl GGT manuell richtig

In [18]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
cols = ["Unique Code", "ResponseTime", "Algorithm", "Flag", "Correctness", "Difficulty", "ClickData"]

def fill(list_data, dataframe):
    for idx, row in dataframe.iterrows():
        response_time = row["response_time"]
        algorithm = row["algo_name"]
        flag = row["flag"]
        click_data = row["click_data"]
        correctness = row["correctness"]
        difficulty = row["difficulty"]
        unique_code = row["unique_code"]
        list_data.append([unique_code, response_time, algorithm, flag, correctness, difficulty, click_data])

data = []
fill(data, df_tmp)
df = pd.DataFrame(data, columns=cols)
df.to_excel("./results/preprocessed_pretest.xlsx", index=False)